In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from pprint import pprint

driver = webdriver.Firefox()

# Open the website
# https://transitfeeds.com/p/rtd-denver/188?p=26
url = 'https://transitfeeds.com/p/rtd-denver/188'  # Replace with the url that you intend, e.g.
# if you dont want rtd-denver then you have to choose something else
driver.get(url)

# Function to extract download links
def get_download_links():
    download_links = []
    
    # Find all "Download" buttons with the class "btn btn-xs btn-primary"
    buttons = driver.find_elements(By.XPATH, '//a[contains(@class, "btn btn-xs btn-primary")]')
    
    for button in buttons:
        # Get the href attribute (download link)
        link = button.get_attribute('href')
        if link:
            download_links.append(link)
    
    return download_links


# Function to navigate to the next page
def go_to_next_page(page_num):
    try:
        # Find the pagination button (adjust the XPath)
        next_page_button = driver.find_element(By.XPATH, f'//a[text()="{page_num}"]')
        next_page_button.click()
        time.sleep(2)  # Wait for the page to load
    except Exception as e:
        print(f"Could not navigate to page {page_num}: {e}")
        return False
    return True

# Start scraping the links
all_download_links = []

# This is a hardcoded number which is bad but ideally
# we should be able to find the last page number rather simply.
# TODO: find last page number instead of 90 
for page in range(1, 90):
    print(f"Scraping page {page}")
    # Get download links from the current page
    all_download_links += get_download_links()
    
    # Navigate to the next page (skip for the last page)
    if page < 90:
        go_to_next_page(page + 1)

# Close the browser
driver.quit()

# Print the collected download links
for link in all_download_links:
    print(link)


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50
Scraping page 51
Scraping page 52
Scraping page 53
Scraping page 54
Scraping page 55
Scraping page 56
Scraping page 57
Scraping page 58
Scraping page 59
Scrapi

In [16]:

pprint(all_download_links)

['https://transitfeeds.com/p/rtd-denver/188/latest/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231228/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231218/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231216/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231215/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231213/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231211/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231209/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231208/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231206/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231023/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231021/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231018/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231016/download',
 'https://transitfeeds.com/p/rtd-denver/188/20231014/download',
 'https://transitfeeds.com/p/rtd-denver/18

In [21]:
import os
import requests

# Directory to save the downloaded files
save_dir = 'denver'
os.makedirs(save_dir, exist_ok=True)

# Function to download a file from a URL and save it to a specified directory
def download_file(url, save_path):
    response = requests.get(url, stream=True, verify=False)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download: {url}")

# Function to extract the part of the URL immediately before "download"
def extract_part_from_url(url):
    return url.split('/')[-2]

# Download the first two links
for link in all_download_links:
    part_str = extract_part_from_url(link)
    file_name = f"gtfs_{part_str}.zip"
    save_path = os.path.join(save_dir, file_name)
    download_file(link, save_path)

/Users/jfleisc1/ENV3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transitfeeds.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/jfleisc1/ENV3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openmobilitydata-data.s3-us-west-1.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

# Distill GTFS

Do not run the following cell until using onebusaway's gtfs merge module.

In [33]:
import os
import shutil
from datetime import datetime

# Directory containing GTFS ZIP files
gtfs_dir = 'denver'
bad_dir = os.path.join(gtfs_dir, 'bad')
extra_dir = os.path.join(gtfs_dir, 'extra')

# Create directories if they don't exist
os.makedirs(bad_dir, exist_ok=True)
os.makedirs(extra_dir, exist_ok=True)

# List of extra years
extra_years = ['2015', '2016', '2017', '2018']

# Function to check if a string is a valid date in the format YYYYMMDD
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, '%Y%m%d')
        return True
    except ValueError:
        return False

# Dictionary to store the earliest file per quarter
earliest_files = {}

# Function to get the quarter from a date string
def get_quarter(date_str):
    year = date_str[:4]
    month = int(date_str[4:6])
    quarter = (month - 1) // 3 + 1
    return f"{year}Q{quarter}"

# Iterate over all GTFS ZIP files in the directory
for file_name in os.listdir(gtfs_dir):
    if file_name.endswith('.zip'):
        file_path = os.path.join(gtfs_dir, file_name)
        date_str = file_name.split('_')[-1].replace('.zip', '')

        # Check if the date part is valid
        if not is_valid_date(date_str):
            shutil.move(file_path, os.path.join(bad_dir, file_name))
            continue

        # Extract year and quarter
        year = date_str[:4]
        quarter = get_quarter(date_str)

        # Check if the year is in the list of extra years
        if year in extra_years:
            shutil.move(file_path, os.path.join(extra_dir, file_name))
            continue

        # Check if this is the earliest file for the quarter
        if quarter not in earliest_files:
            earliest_files[quarter] = (date_str, file_name)
        else:
            current_earliest_date, current_earliest_file = earliest_files[quarter]
            if date_str < current_earliest_date:
                # Move the current earliest file to extra
                shutil.move(os.path.join(gtfs_dir, current_earliest_file), os.path.join(extra_dir, current_earliest_file))
                # Update with the new earliest file
                earliest_files[quarter] = (date_str, file_name)
            else:
                # Move the current file to extra
                shutil.move(file_path, os.path.join(extra_dir, file_name))

print("Processing complete.")

Processing complete.


# GraphQL Interface

Do not run the following until the OTP instance is live.

# OTP GraphQL

In [38]:
import requests
import json

url = 'http://localhost:9999/otp/gtfs/v1'

#date: "2021-09-27"
# date: "2024-10-02"
query = """
query {
  plan(
    fromPlace: "39.79062,-104.9114"
    toPlace: "39.68415,-105.0370"
    date: "2021-09-27"
    time: "15:23:10"
    numItineraries: 10
    transportModes: [{ mode: WALK }, { mode: BUS }, { mode: RAIL }, { mode: TRAM }]
  ) {
    itineraries {
      duration
      legs {
        startTime
        endTime
        mode
        distance
        from {
          name
        }
        to {
          name
        }
      }
    }
  }
}
"""

payload = {
    "query": query,
    
}

headers = {
    'Content-Type': 'application/json',
    'OTPTimeout': '180000'
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))
else:
    print(f"Error: {response.status_code}, {response.text}")

# lastly 
# TODO: find shortest duration and convert that to a human readable format


{
  "data": {
    "plan": {
      "itineraries": [
        {
          "duration": 5779,
          "legs": [
            {
              "startTime": 1632778721000,
              "endTime": 1632779038000,
              "mode": "WALK",
              "distance": 403.63,
              "from": {
                "name": "Origin"
              },
              "to": {
                "name": "50th Ave & Niagara St"
              }
            },
            {
              "startTime": 1632779038000,
              "endTime": 1632779700000,
              "mode": "BUS",
              "distance": 4275.83,
              "from": {
                "name": "50th Ave & Niagara St"
              },
              "to": {
                "name": "Central Park Station Gate D2"
              }
            },
            {
              "startTime": 1632779700000,
              "endTime": 1632779934000,
              "mode": "WALK",
              "distance": 217.95,
              "from": {
               